<a href="https://colab.research.google.com/github/Nguyendai202/VN-text-classification-/blob/main/VN_text_classification_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import Thư Viện**

In [ ]:
!pip install pyvi 
!pip install gensim

In [3]:
from pyvi import ViTokenizer, ViPosTagger
import numpy as np
import pandas as pd
import gensim
import smart_open
import sklearn
import tensorflow as tf
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.layers import *
from keras.layers import *
from keras.preprocessing.text import Tokenizer
from keras.models import Model
from keras.metrics import sparse_categorical_accuracy
from tensorflow.keras.optimizers import Adam
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn import decomposition, ensemble
import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import os 
from tqdm import tqdm
from tensorflow.keras.preprocessing.sequence import pad_sequences

# **Data Preparation**(tiền xử lí dữ liệu như loại bỏ kí tự đặc biệt , từ không cần thiết , chữ hoa , tách từ ..)

In [ ]:
import os 
dir_path = os.path.dirname(os.path.realpath(os.getcwd()))
dir_path = os.path.join(dir_path, 'Data')
# '/Users/macos/Desktop/Github/NLP/Text Classifier'
# Load data from dataset folder
# VNTC-master/Data/10Topics/Ver1.1/Train_Full
# VNTC-master/Data/10Topics/Ver1.1/Test_Full
def get_data(folder_path):
    X = []
    y = []
    dirs = os.listdir(folder_path)
    for path in tqdm(dirs):
        file_paths = os.listdir(os.path.join(folder_path, path))
        for file_path in tqdm(file_paths):
            with open(os.path.join(folder_path, path, file_path), 'r', encoding="utf-16") as f:
                lines = f.readlines()
                lines = ' '.join(lines)
                lines = gensim.utils.simple_preprocess(lines)# xoá các kí tự đặc biệt như chấm , phẩy đóng mở ngoặc 
                lines = ' '.join(lines)
                # lines = stopwords.words(lines)
                lines = ViTokenizer.tokenize(lines)

#                 sentence = ' '.join(words)
#                 print(lines)
                X.append(lines)
                y.append(path)
#             break
#         break
    return X, y

train_path = os.path.join(dir_path, '/content/drive/MyDrive/NLP/text-classfication/data-test/Test_Full')
X_train, y_train = get_data(train_path)


In [ ]:
# lưu lại và tải về sau dùng , đỡ phải xử lí lại, X là văn bản đã xử lí , y là nhãn 
import pickle
pickle.dump(X_train, open('X_test.pkl', 'wb'))
pickle.dump(y_train, open('y_test.pkl', 'wb'))

# **Feature Engineering**( mã hoá các dữ liệu ta vừa tiền xử lí xong>> vector dạng số >> để đưa vào model train) 

In [27]:
# load dữ liệu 
import pickle  # load dữ liệu 

X_train = pickle.load(open('/content/drive/MyDrive/NLP/text-classfication/pickle-data/train/X_train.pkl', 'rb'))
y_train= pickle.load(open('/content/drive/MyDrive/NLP/text-classfication/pickle-data/train/y_train.pkl', 'rb'))

X_test = pickle.load(open('/content/drive/MyDrive/NLP/text-classfication/pickle-data/test/X_test.pkl', 'rb'))
y_test = pickle.load(open('/content/drive/MyDrive/NLP/text-classfication/pickle-data/test/y_test.pkl', 'rb'))

TF-IDF ( PP thống kê này nhằm phản ánh độ quan trọng của mỗi từ hay n-gram(chuỗi từ trong n kích thước )  từ trên toàn bộ tài liêụ đầu vào , loại pp này có nhược điểm là ko biểu diễn đc nghĩa của từ ) 

In [28]:
def tfidf(data):
  tfidf_vect_ngram = TfidfVectorizer(analyzer='word', max_features=30000, ngram_range=(1, 2))
  tfidf_vect_ngram.fit(data)
  X_data_tfidf_ngram =  tfidf_vect_ngram.transform(data)
  return X_data_tfidf_ngram

In [29]:
x_train_tfidf = tfidf(X_train)
x_test_tfidf = tfidf(X_test)
svd = TruncatedSVD(n_components=300, random_state=42,)# giảm chiều dữ liệu 
X_data_tfidf_ngram_dimen =svd.fit(x_train_tfidf).transform(x_train_tfidf)
X_test_tfidf_ngram_dimen= svd.fit(x_test_tfidf).transform(x_test_tfidf)

In [ ]:
print(x_train_tfidf.get_shape)

word2vec

In [ ]:

from gensim.models import Word2Vec, KeyedVectors
import os 
word2vec_model_path = '/content/drive/MyDrive/NLP/text-classfication/wordvectors-master/baomoi.vn.model.bin'
model = KeyedVectors.load_word2vec_format(word2vec_model_path,binary=True,unicode_errors='igore')
vocab = model.wv.vocab
wv= model.wv


In [ ]:
# mã hoá 
def get_word2vec_data(X):
    word2vec_data = []
    for x in X:
        sentence = []
        for word in x.split(" "):
            if word in vocab:
                print(word)
                sentence.append(wv[word])

        word2vec_data.append(sentence)
#         break
    return get_word2vec_data

X_data_w2v = get_word2vec_data(X_train)
X_test_w2v = get_word2vec_data(X_test)

Bag of words

In [ ]:
def bow(data):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(data)
  databow = tokenizer.texts_to_sequences(data)
  databow= pad_sequences(databow, maxlen=300)
  return databow

In [ ]:
databow_train = bow(X_train)
databow_test = bow(X_test)
svd = TruncatedSVD(n_components=300, random_state=42,)
X_data_bow_dimen =svd.fit(x_train_tfidf).transform(x_train_tfidf)
X_test_bow_dimen= svd.fit(x_test_tfidf).transform(x_test_tfidf)

**CÁC PHƯƠNG PHÁP HỌC MÁY DÙNG ĐỂ PHÂN LOẠI ***

Convert y to categorical

In [12]:
encoder = preprocessing.LabelEncoder()
y_train_n = encoder.fit_transform(y_train)
y_test_n = encoder.fit_transform(y_test)

In [13]:
encoder.classes_

array(['Chinh tri Xa hoi', 'Doi song', 'Kinh doanh'], dtype='<U16')

Hàm train model 

In [31]:
from sklearn.model_selection import train_test_split

In [40]:
def train_model(classifier, X_data, y_data, X_test, y_test, is_neuralnet=False, n_epochs=5000):       
    X_train, X_val, y_train, y_val = train_test_split(X_data, y_data, test_size=0.1, random_state=42)
    
    if is_neuralnet:
        classifier.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=n_epochs, batch_size=512)
        
        val_predictions = classifier.predict(X_val)
        test_predictions = classifier.predict(X_test)
        val_predictions = val_predictions.argmax(axis=-1)
        test_predictions = test_predictions.argmax(axis=-1)
    else:
        classifier.fit(X_train, y_train)
    
        train_predictions = classifier.predict(X_train)
        val_predictions = classifier.predict(X_val)
        test_predictions = classifier.predict(X_test)
        
    print("Validation accuracy: ", metrics.accuracy_score(val_predictions, y_val))
    print("Test accuracy: ", metrics.accuracy_score(test_predictions, y_test))
    print(classification_report(test_predictions, y_test))

SVM

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_data_tfidf_ngram_dimen, y_train, test_size=0.2, random_state=42)

In [34]:
clf=svm.SVC(kernel='linear',C=100)
clf.fit(X_train, y_train)
val_predictions = clf.predict(X_test_tfidf_ngram_dimen)
print(classification_report(val_predictions, y_test)) 

                  precision    recall  f1-score   support

Chinh tri Xa hoi       0.68      0.63      0.65      8238
        Doi song       0.86      0.50      0.63      3484
      Kinh doanh       0.34      0.58      0.43      3159

        accuracy                           0.59     14881
       macro avg       0.63      0.57      0.57     14881
    weighted avg       0.65      0.59      0.60     14881



NAIVE BAYES

In [ ]:
train_model(naive_bayes.BernoulliNB(),X_data_tfidf_ngram_dimen,y_train,X_test_tfidf_ngram_dimen,y_test,is_neuralnet=False)

Validation accuracy:  0.8392694063926941
Test accuracy:  0.4164370674013843
['Doi song' 'Chinh tri Xa hoi' 'Chinh tri Xa hoi' ... 'Chinh tri Xa hoi'
 'Doi song' 'Kinh doanh']
